# Data Science Final Project - EE 379k - sp18 - 

# Shahshank Kambhampati - skk834, Shrikara Murthy - svm456, Pranav Harathi - , Neil Charles - 

# Job Satisfaction Prediction

## Imports

### All hail lord and savior XGB

In [46]:
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from scipy import stats
from sklearn.linear_model import RidgeCV, LassoCV, LogisticRegression, Ridge, Lasso, HuberRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomTreesEmbedding, AdaBoostClassifier, AdaBoostRegressor, RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import LeaveOneOut, train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import RFE,RFECV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.feature_selection import SelectKBest,  chi2
from sklearn.svm import SVR

from __future__ import print_function
%config inlinebackend.figure_format = 'retina' 
%matplotlib inline

# Initial Observations about the Data and Pre-Processing

In [47]:
number = re.compile('[\d,]+')
def get_first_number(val):
    matched = number.match(str(val))
    if matched:
        return int(matched.group())
    else:
        return np.nan

satisfaction_strs = {
    'I love my job': 10,
    'I\'m somewhat satisfied with my job': 7.5,
    'I\'m neither satisfied nor dissatisfied with my job': 5,
    'I\'m somewhat dissatisfied with my job': 2.5,
    'I hate my job': 0,
}

binary_labels = [
    'Lang & Tech',
    'Training & Education',
    'How can companies improve interview process',
    'Why try Stack Overflow Careers',
    'Most important aspect of new job opportunity',
    'Most annoying about job search',
    'Appealing message traits',
    'Most urgent info about job opportunity',
    'Who do you want to communicate with about a new job opportunity',
    'Why use Stack Overflow',
    'Why answer',
    'Source control used',
]

numeric_labels = [
    'Age',
    'Years IT / Programming Experience',
    'Compensation: midpoint'
]

yes_no_labels = [
    'Changed Jobs in last 12 Months'
]


In [48]:
# 2015 preproc
data = pd.read_csv('data/2015.csv')
to_drop = [label for label in data if 'write-in' in label.lower()]

data['Job Satisfaction'] = data['Job Satisfaction']\
                                    .map(satisfaction_strs).astype('float')
data = data[data['Job Satisfaction'].notnull()]
to_drop.append('Country')
to_drop.append('Compensation')

data['Age'] = data['Age'].map(get_first_number).astype('float')
data['gender_M'] = (data['Gender'] == 'Male').astype('int8')
data['gender_F'] = (data['Gender'] == 'Female').astype('int8')
to_drop.append('Gender')
to_drop.append('Prefered Source Control')

bin_labels = [key for key in data if any(label in key for label in binary_labels)]
data[bin_labels] = data[bin_labels].apply(lambda col: col.notnull().astype('int8'))

data[numeric_labels] = data[numeric_labels].applymap(get_first_number)
data[yes_no_labels] = data[yes_no_labels]\
                                .apply(lambda col: col.map({'Yes': 1, 'No': 0}))\
                                .fillna(0)

data.drop(to_drop, axis=1, inplace=True)
data = pd.get_dummies(data)

print("Columns with NaNs:")

for key in data:
    if data[key].isnull().any():
        print (key)
data.fillna(data.mean(), inplace=True)
print ('Filling with mean of column')

Columns with NaNs:
Age
Years IT / Programming Experience
Compensation: midpoint
Filling with mean of column


In [49]:
# data contains a DataFrame with no NaNs, all numbers.
# We're trying to predict the "Job Satisfaction" column.

In [50]:
#X = data[['Compensation: midpoint','Purchasing Power_I have no say in purchasing what I need or want at work','Remote Status_Never','Changed Jobs in last 12 Months']]
X = data.drop('Job Satisfaction',axis=1)
y = data['Job Satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)#stratify=y
scoring = {'mean': make_scorer(mean_squared_error)}

In [51]:
print (data.head())

    Age  Years IT / Programming Experience  Current Lang & Tech: Android  \
0  25.0                                2.0                             0   
1  20.0                                1.0                             0   
2  20.0                                1.0                             0   
3  25.0                                6.0                             0   
4  30.0                                2.0                             0   

   Current Lang & Tech: Arduino  Current Lang & Tech: AngularJS  \
0                             0                               0   
1                             0                               0   
2                             0                               0   
3                             0                               0   
4                             0                               0   

   Current Lang & Tech: C  Current Lang & Tech: C++  \
0                       0                         0   
1                       0     

In [11]:
print (data.describe())

                Age  Years IT / Programming Experience  \
count  16004.000000                       16004.000000   
mean      28.478348                           5.839325   
std        6.955481                           3.850501   
min       20.000000                           1.000000   
25%       25.000000                           2.000000   
50%       25.000000                           6.000000   
75%       30.000000                          11.000000   
max       51.000000                          11.000000   

       Current Lang & Tech: Android  Current Lang & Tech: Arduino  \
count                  16004.000000                  16004.000000   
mean                       0.173269                      0.061110   
std                        0.378492                      0.239539   
min                        0.000000                      0.000000   
25%                        0.000000                      0.000000   
50%                        0.000000                      0.0000

In [43]:
c = data.corr()
print(c['Job Satisfaction'].sort_values())

Open to new job opportunities_I am actively looking for a new job                          -0.261420
Open to new job opportunities_I am open to new job opportunities                           -0.176831
Purchasing Power_I have no say in purchasing what I need or want at work                   -0.164541
Remote Status_Never                                                                        -0.137992
Perception of recruiter contact_I wish more recruiters contacted me!                       -0.135077
Most annoying about job search: Finding job I'm qualified for                              -0.096418
Perception of contact form: Email_Great                                                    -0.091982
Perception of contact form: Stack Overflow Careers_Great                                   -0.087447
Why try Stack Overflow Careers: Jobs site for programmers                                  -0.078943
Appealing message traits: Salary information                                               

In [ ]:
model = xgb.XGBRegressor()
rfe = RFE(model, 50,2)
rfe = rfe.fit(X_train,y_train)
X_train = rfe.transform(X_train)
X_test = rfe.transform(X_test)
#rfe_preds = rfe.predict(X_test)
#print mean_squared_error(y_test,rfe_preds)


In [ ]:
parameters = {
    'n_estimators':[160]
}
modelETC = ExtraTreesRegressor()
gridETC = GridSearchCV(modelETC, param_grid=parameters,scoring = scoring, cv=5, refit = 'mean',verbose=1)
gridETC.fit(X_train,y_train)
print(gridETC.best_params_)
#print(modelETC.feature_importances_)
etc_preds = gridETC.predict(X_test)
print (mean_squared_error(y_test,etc_preds))

In [ ]:
print (mean_squared_error(y_test,etc_preds))

In [6]:
xgb_model = xgb.XGBRegressor()
#gridXGB = GridSearchCV(xgb_model, param_grid=parameters,scoring = scoring, cv=3, verbose=2, refit = 'mean')
#gridXGB.fit(X_train, y_train,eval_metric='rmse')
#print(gridXGB.best_params_)
#xgb_preds = gridXGB.predict(X_test)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
print (mean_squared_error(y_test,xgb_preds)**.5)
#xgb_copy = list(xgb_preds)
xgb_model.fit(X, y)
xgb_model._Booster.save_model('output2015.model')
xgb.plot_importance(xgb_model,max_num_features = 10)


2.471834236419676


In [13]:
xgb_model._Booster.predict(xgb.DMatrix([0.0,0.0,0.0,0.0], feature_names=[u'Compensation: midpoint', u'Purchasing Power_I have no say in purchasing what I need or want at work', u'Remote Status_Never', u'Changed Jobs in last 12 Months']))

array([7.33352], dtype=float32)

In [19]:
parameters = { #when use hyperthread, xgboost may become slower
              'booster':['gbtree'],# dart is nice but far too slow
              'learning_rate': [.01], #
              'max_depth': [7],#1,3,5,6,7,8,10
              'min_child_weight': [8],#1,3,5,7,8,9,10
              'reg_alpha':[.01],#.0001,.001,.01,.1,1,10
              'silent': [0],
              'subsample': [.6],#.6,.7,.8,.9
              'gamma':[.001],#.1,.01,.001.,.005,0
              'colsample_bytree': [.8],#.6,.7,.8,.9
              'n_estimators': [10000], 
              'seed': [1337]
}

In [20]:
xgb_model = xgb.XGBRegressor()
gridXGB = GridSearchCV(xgb_model, param_grid=parameters,scoring = scoring, cv=3, verbose=2, refit = 'mean')
gridXGB.fit(X_train, y_train,eval_metric='rmse')
#print(gridXGB.best_params_)
xgb_preds = gridXGB.predict(X_test)
#xgb_model.fit(X_train, y_train)
#xgb_preds = xgb_model.predict(X_test)
print (mean_squared_error(y_test,xgb_preds)**.5)
#xgb_copy = list(xgb_preds)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] reg_alpha=0.01, colsample_bytree=0.8, silent=0, learning_rate=0.01, min_child_weight=8, n_estimators=10000, subsample=0.6, seed=1337, max_depth=7, gamma=0.001, booster=gbtree 
[CV]  reg_alpha=0.01, colsample_bytree=0.8, silent=0, learning_rate=0.01, min_child_weight=8, n_estimators=10000, subsample=0.6, seed=1337, max_depth=7, gamma=0.001, booster=gbtree, total=21.1min
[CV] reg_alpha=0.01, colsample_bytree=0.8, silent=0, learning_rate=0.01, min_child_weight=8, n_estimators=10000, subsample=0.6, seed=1337, max_depth=7, gamma=0.001, booster=gbtree 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 21.4min remaining:    0.0s


[CV]  reg_alpha=0.01, colsample_bytree=0.8, silent=0, learning_rate=0.01, min_child_weight=8, n_estimators=10000, subsample=0.6, seed=1337, max_depth=7, gamma=0.001, booster=gbtree, total=25.0min
[CV] reg_alpha=0.01, colsample_bytree=0.8, silent=0, learning_rate=0.01, min_child_weight=8, n_estimators=10000, subsample=0.6, seed=1337, max_depth=7, gamma=0.001, booster=gbtree 
[CV]  reg_alpha=0.01, colsample_bytree=0.8, silent=0, learning_rate=0.01, min_child_weight=8, n_estimators=10000, subsample=0.6, seed=1337, max_depth=7, gamma=0.001, booster=gbtree, total=21.7min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 68.6min finished


2.2666836788652214


In [9]:
print(gridXGB.best_params_)

NameError: name 'gridXGB' is not defined

In [17]:
print (mean_squared_error(y_test,xgb_preds)**.5)


2.322300794571225


In [47]:
print (sum(y_test)/len(y_test))
print (sum(xgb_preds)/len(xgb_preds))

7.5
7.492633136249195


In [44]:
parameters = {
    'n_estimators':[1],
    'learning_rate':[.1]
   # 'base_estimator':[ExtraTreesClassifier(160,class_weight='balanced')]
   # 'base_estimator':[grid]
}
modelABC = AdaBoostRegressor(base_estimator=xgb.XGBRegressor(reg_alpha=.01, colsample_bytree=.8, 
                                silent=0, learning_rate=.01, min_child_weight=8, n_estimators=10000, 
                                            subsample=.6, max_depth=7,gamma=.001,booster='gbtree'),n_estimators =50)
#gridABC = GridSearchCV(modelABC, param_grid=parameters,scoring = scoring, cv=2, refit = 'mean',verbose=2)
modelABC.fit(X_train,y_train)
abc_preds = modelABC.predict(X_test)
print (mean_squared_error(y_test,abc_preds)**.5)
#print(gridABC.best_params_)
#print(model.feature_importances_)
#best score 2.2432 - all features

2.580892773519512


In [48]:
def clipS(preds):
    for pred, element in enumerate(preds):
        if preds[pred] < 1.25:
            preds[pred] = 0
        elif preds[pred] < 3.75:
            preds[pred] = 2.5
        elif  preds[pred] < 6.25:
            preds[pred] = 5
        elif  preds[pred] < 8.75:
            preds[pred] = 7.5
        else: 
            preds[pred] = 10
    return preds

In [49]:
# for pred, clipped, actual in zip(xgb_preds, clipS(xgb_copy), y_test ):
#     print (pred, clipped, actual)

8.286079 7.5 10.0
7.927208 7.5 7.5
6.8665676 7.5 7.5
7.365894 7.5 7.5
8.49939 7.5 5.0
7.1254463 7.5 5.0
6.702565 7.5 7.5
5.842753 5 7.5
8.738859 7.5 10.0
7.841649 7.5 10.0
6.74351 7.5 7.5
7.807215 7.5 7.5
9.261417 10 7.5
7.109841 7.5 5.0
6.8010564 7.5 7.5
7.597346 7.5 10.0
6.9410725 7.5 2.5
6.5449824 7.5 0.0
7.402582 7.5 5.0
6.431654 7.5 0.0
7.1956844 7.5 7.5
7.6999984 7.5 10.0
6.1977854 5 2.5
7.982983 7.5 10.0
7.190724 7.5 7.5
6.7932687 7.5 10.0
8.369915 7.5 7.5
8.421787 7.5 7.5
7.026611 7.5 7.5
6.857605 7.5 7.5
8.973689 10 10.0
6.7475266 7.5 2.5
8.897934 10 7.5
7.8295035 7.5 10.0
9.146121 10 10.0
8.495934 7.5 7.5
6.728901 7.5 7.5
7.36117 7.5 10.0
8.701159 7.5 10.0
7.6483 7.5 10.0
7.262645 7.5 7.5
7.3512554 7.5 7.5
7.0285273 7.5 10.0
8.729461 7.5 7.5
7.8325706 7.5 7.5
8.742565 7.5 10.0
6.331672 7.5 0.0
8.884819 10 7.5
7.004472 7.5 7.5
7.0543513 7.5 5.0
7.403653 7.5 10.0
7.0553303 7.5 7.5
7.1007104 7.5 7.5
4.405138 5 5.0
6.3311124 7.5 5.0
7.363051 7.5 5.0
6.0551515 5 2.5
9.117585 10 10

7.2498374 7.5 10.0
7.0629873 7.5 10.0
6.8996253 7.5 7.5
6.8121514 7.5 10.0
7.2658896 7.5 2.5
8.852533 10 10.0
7.584811 7.5 5.0
8.762749 10 10.0
6.7032113 7.5 7.5
7.040448 7.5 7.5
8.731287 7.5 10.0
8.339863 7.5 10.0
9.299665 10 10.0
7.84971 7.5 7.5
5.7796865 5 7.5
5.215879 5 2.5
6.0454664 5 5.0
7.069505 7.5 7.5
7.026026 7.5 10.0
8.7816305 10 7.5
7.6395483 7.5 10.0
8.811912 10 7.5
7.073048 7.5 7.5
6.7585797 7.5 7.5
5.5238676 5 7.5
7.074044 7.5 7.5
8.605527 7.5 10.0
4.81072 5 2.5
6.474626 7.5 10.0
7.489467 7.5 10.0
6.297437 7.5 7.5
9.015276 10 10.0
7.1141887 7.5 2.5
8.855669 10 7.5
7.5748086 7.5 7.5
7.4217353 7.5 10.0
7.6471562 7.5 7.5
6.816087 7.5 5.0
8.472724 7.5 10.0
6.552516 7.5 7.5
7.1472735 7.5 2.5
5.7331457 5 7.5
6.8766747 7.5 2.5
5.6829224 5 2.5
9.096558 10 10.0
7.444388 7.5 7.5
8.846415 10 7.5
7.449927 7.5 7.5
7.189878 7.5 7.5
6.0751224 5 10.0
7.364566 7.5 10.0
5.6063213 5 10.0
7.286406 7.5 7.5
7.6666327 7.5 7.5
6.3354225 7.5 10.0
5.25561 5 10.0
9.825521 10 10.0
6.786795 7.5 10.0

6.7574396 7.5 7.5
7.146657 7.5 10.0
8.529603 7.5 5.0
7.1334844 7.5 7.5
5.8378696 5 2.5
7.2832446 7.5 7.5
7.612223 7.5 7.5
6.818888 7.5 7.5
7.420518 7.5 7.5
7.2980275 7.5 5.0
5.7383604 5 2.5
6.2615175 7.5 7.5
6.6522627 7.5 7.5
6.1226707 5 7.5
4.436254 5 7.5
7.765819 7.5 7.5
7.6464105 7.5 5.0
7.4550295 7.5 10.0
8.247253 7.5 7.5
7.783889 7.5 10.0
6.8527164 7.5 7.5
7.6722775 7.5 7.5
7.215664 7.5 2.5
6.639135 7.5 10.0
7.11952 7.5 7.5
8.538041 7.5 10.0
9.917143 10 10.0
7.593615 7.5 10.0
8.31717 7.5 7.5
7.4880567 7.5 2.5
6.509329 7.5 7.5
6.3432565 7.5 2.5
8.837619 10 10.0
7.251931 7.5 7.5
5.1459727 5 2.5
4.7939253 5 0.0
7.554099 7.5 10.0
5.7530193 5 2.5
7.1315784 7.5 10.0
6.6707196 7.5 7.5
9.495126 10 10.0
7.509618 7.5 10.0
7.072568 7.5 10.0
8.936038 10 10.0
7.7594514 7.5 7.5
7.373159 7.5 10.0
7.533154 7.5 10.0
9.063518 10 7.5
6.768366 7.5 2.5
6.259156 7.5 7.5
7.2441187 7.5 7.5
7.7757993 7.5 7.5
6.8099904 7.5 5.0
6.2015033 5 2.5
6.786014 7.5 5.0
8.877727 10 10.0
8.141465 7.5 5.0
8.738115 7.5 

5.795179 5 2.5
8.171051 7.5 5.0
5.4485793 5 2.5
6.559353 7.5 10.0
8.21508 7.5 5.0
8.887403 10 10.0
6.473309 7.5 5.0
9.495146 10 10.0
6.067431 5 7.5
6.878742 7.5 7.5
7.1737523 7.5 5.0
7.470629 7.5 7.5
7.115641 7.5 0.0
5.6777554 5 5.0
5.8339877 5 2.5
7.111055 7.5 7.5
6.7236257 7.5 10.0
7.230857 7.5 2.5
6.8952537 7.5 5.0
7.876106 7.5 10.0
9.333084 10 7.5
6.7052875 7.5 10.0
8.9069605 10 7.5
9.08294 10 10.0
8.777982 10 10.0
8.99087 10 10.0
7.303084 7.5 10.0
7.020656 7.5 7.5
5.3853273 5 5.0
8.794083 10 10.0
8.935527 10 10.0
7.2178936 7.5 7.5
8.520957 7.5 5.0
7.0320234 7.5 2.5
8.815841 10 7.5
9.146775 10 10.0
9.14814 10 10.0
6.1299076 5 0.0
8.279397 7.5 7.5
8.929781 10 10.0
7.00511 7.5 10.0
7.3344545 7.5 10.0
7.4628363 7.5 10.0
6.5345902 7.5 5.0
7.4571605 7.5 7.5
5.4562893 5 7.5
8.557975 7.5 7.5
7.588536 7.5 7.5
8.936083 10 10.0
7.102791 7.5 5.0
8.20962 7.5 10.0
7.629561 7.5 10.0
6.944566 7.5 5.0
7.9404235 7.5 7.5
6.6885858 7.5 7.5
7.023529 7.5 7.5
5.356553 5 2.5
9.079288 10 10.0
7.165941 7.5

7.760685 7.5 10.0
8.821927 10 7.5
7.4442616 7.5 7.5
7.7990084 7.5 7.5
6.3827243 7.5 2.5
7.2358866 7.5 10.0
8.875368 10 10.0
6.5619397 7.5 5.0
9.182947 10 10.0
4.367136 5 7.5
7.6697392 7.5 7.5
7.6056943 7.5 7.5
7.1915956 7.5 10.0
8.1389675 7.5 10.0
8.704581 7.5 7.5
7.4515715 7.5 7.5
7.3995833 7.5 2.5
5.835661 5 2.5
8.826144 10 10.0
9.235827 10 7.5
5.5191054 5 5.0
6.203065 5 7.5
6.5741863 7.5 7.5
6.2972455 7.5 10.0
7.064277 7.5 5.0
5.743789 5 5.0
8.2548685 7.5 10.0
8.869461 10 10.0
7.3615146 7.5 7.5
6.889896 7.5 7.5
6.8485804 7.5 7.5
6.5264893 7.5 7.5
6.994022 7.5 7.5
7.7897563 7.5 10.0
8.830721 10 7.5
6.8106136 7.5 10.0
7.553045 7.5 10.0
6.657207 7.5 5.0
7.0640388 7.5 7.5
6.8612704 7.5 5.0
8.883724 10 7.5
7.7433324 7.5 7.5
7.1755404 7.5 10.0
7.748843 7.5 7.5
6.334149 7.5 5.0
9.155457 10 10.0
8.379332 7.5 10.0
8.09853 7.5 10.0
8.055823 7.5 7.5
6.5188313 7.5 10.0
8.244404 7.5 7.5
7.3874316 7.5 5.0
7.6197114 7.5 7.5
5.0924687 5 2.5
7.3565054 7.5 7.5
6.8516836 7.5 5.0
7.22283 7.5 7.5
8.5440

In [51]:
print (mean_squared_error(y_test,clipS(xgb_preds))**.5)

2.35883188889119


In [25]:
import tensorflow as tf
import keras
import theano
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# tf.logging.set_verbosity(tf.logging.INFO)
np.random.seed(123)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

np.random.seed(1337)  # for reproducibility

/home/shrikar/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [37]:
model = Sequential()
model.add(Dense(4, input_dim=4, activation='relu'))
#model.add(Dense(200, activation='sigmoid'))
model.add(Dense(2, activation='hard_sigmoid'))
#model.add(Dense(50, activation='hard_sigmoid'))
#model.add(Dense(40, activation='sigmoid'))
#model.add(Dense(10, activation='sigmoid'))
#model.add(Dense(5, activation='sigmoid'))
model.add(Dense(1, activation='relu'))

model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=.001))

model.fit(X_train, y_train, epochs=200, batch_size=10)

nn_preds = model.predict(X_test)


Epoch 1/200
11202/11202 [==============================] - 2s 185us/step - loss: 41.4869
Epoch 2/200
11202/11202 [==============================] - 2s 178us/step - loss: 22.5995
Epoch 3/200
11202/11202 [==============================] - 2s 165us/step - loss: 12.4291
Epoch 4/200
11202/11202 [==============================] - 2s 158us/step - loss: 7.9893
Epoch 5/200
11202/11202 [==============================] - 1s 113us/step - loss: 6.6835
Epoch 6/200
11202/11202 [==============================] - 1s 106us/step - loss: 6.4924
Epoch 7/200
11202/11202 [==============================] - 1s 106us/step - loss: 6.4853
Epoch 8/200
11202/11202 [==============================] - 1s 120us/step - loss: 6.4852
Epoch 9/200
11202/11202 [==============================] - 1s 108us/step - loss: 6.4849
Epoch 10/200
11202/11202 [==============================] - 1s 107us/step - loss: 6.4851
Epoch 11/200
11202/11202 [==============================] - 1s 125us/step - loss: 6.4850
Epoch 12/200
11202/11202 [=

Epoch 93/200
11202/11202 [==============================] - 1s 133us/step - loss: 6.4853
Epoch 94/200
11202/11202 [==============================] - 1s 115us/step - loss: 6.4850
Epoch 95/200
11202/11202 [==============================] - 1s 108us/step - loss: 6.4843
Epoch 96/200
11202/11202 [==============================] - 1s 116us/step - loss: 6.4840
Epoch 97/200
11202/11202 [==============================] - 1s 109us/step - loss: 6.4848
Epoch 98/200
11202/11202 [==============================] - 1s 130us/step - loss: 6.4852
Epoch 99/200
11202/11202 [==============================] - 2s 149us/step - loss: 6.4845
Epoch 100/200
11202/11202 [==============================] - 2s 171us/step - loss: 6.4856
Epoch 101/200
11202/11202 [==============================] - 1s 131us/step - loss: 6.4848
Epoch 102/200
11202/11202 [==============================] - 2s 175us/step - loss: 6.4850
Epoch 103/200
11202/11202 [==============================] - 1s 129us/step - loss: 6.4841
Epoch 104/200
112

11202/11202 [==============================] - 1s 122us/step - loss: 6.4849
Epoch 184/200
11202/11202 [==============================] - 1s 128us/step - loss: 6.4849
Epoch 185/200
11202/11202 [==============================] - 1s 125us/step - loss: 6.4851
Epoch 186/200
11202/11202 [==============================] - 2s 138us/step - loss: 6.4842
Epoch 187/200
11202/11202 [==============================] - 2s 136us/step - loss: 6.4849
Epoch 188/200
11202/11202 [==============================] - 2s 140us/step - loss: 6.4843
Epoch 189/200
11202/11202 [==============================] - 2s 159us/step - loss: 6.4849
Epoch 190/200
11202/11202 [==============================] - 2s 163us/step - loss: 6.4851
Epoch 191/200
11202/11202 [==============================] - 2s 157us/step - loss: 6.4847
Epoch 192/200
11202/11202 [==============================] - 1s 114us/step - loss: 6.4853
Epoch 193/200
11202/11202 [==============================] - 1s 117us/step - loss: 6.4852
Epoch 194/200
11202/1120

In [35]:
print (mean_squared_error(y_test,nn_preds)**.5)

NameError: name 'nn_preds' is not defined

In [15]:
print (mean_absolute_error(y_test,nn_preds)**.5)

1.918158279204034


In [10]:
# Refer https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
# Refer https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# Aim of this is to show case use of keras and grid search libraries
print("Hello World!")
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import numpy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD
from keras.constraints import maxnorm

##############################################################
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


def baseline_model():
    model = Sequential()
    model.add(Dense(346, input_dim=346, activation='relu'))
    model.add(Dense(200, activation=activation))
    model.add(Dense(100, activation=activation))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

print("Hello World!")

estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=100, verbose=1)

# Use scikit-learn to grid search 
#activation =  ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'softsign'] # softmax, softplus, softsign 
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
learn_rate = [.00001,.0001,0.001, 0.01, 0.1, 0.3]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
weight_constraint=[1, 2, 3, 4, 5]
neurons = [1, 5, 10, 15, 20, 25, 30]
init = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
optimizer = [ 'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']


# grid search epochs, batch size
#learn_rate = [.00001,.0001, 0.001]
activations =  ['relu', 'hard_sigmoid', 'softsign'] # softmax, softplus, softsign 
#dropout_rate = [0.0, 0.3, 0.7,]
epochs = [1, 10,50,150] # add 50, 100, 150 etc
batch_size = [20,50,100] # add 5, 10, 20, 40, 60, 80, 100 etc
param_grid = dict(epochs=epochs, batch_size=batch_size, activation=activations )

#gridSearch
grid = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=-1, verbose = 1)
grid_result = grid.fit(X_test, y_test) 


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Hello World!
Hello World!
Fitting 3 folds for each of 36 candidates, totalling 108 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/shrikar/anaconda2/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/home/shrikar/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

......................................

In [52]:
print (grid.best_params_)
ngrid_preds = grid.predict(X_test)
print (mean_squared_error(y_test,ngrid_preds)**.5)

NameError: name 'grid' is not defined

In [64]:
parameters = { #when use hyperthread, xgboost may become slower
              'epsilon':[1.1,2,10,100,1000,10000],# dart is nice but far too slow
              'max_iter':[10,100,1000,10000],
              'alpha':[.00001,.0001,.001,.01,.1,1,10,100,1000],
}
hr_model = HuberRegressor()

gridHR = GridSearchCV(hr_model, param_grid=parameters,scoring = scoring, cv=3, verbose=2, refit = 'mean')
gridHR.fit(X_train,y_train)
hr_preds = gridHR.predict(X_test)
print (mean_squared_error(y_test,hr_preds)**.5)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] alpha=1e-05, max_iter=10, epsilon=1.1 ...........................
[CV] ............ alpha=1e-05, max_iter=10, epsilon=1.1, total=   0.3s
[CV] alpha=1e-05, max_iter=10, epsilon=1.1 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ............ alpha=1e-05, max_iter=10, epsilon=1.1, total=   0.3s
[CV] alpha=1e-05, max_iter=10, epsilon=1.1 ...........................
[CV] ............ alpha=1e-05, max_iter=10, epsilon=1.1, total=   0.3s
[CV] alpha=1e-05, max_iter=100, epsilon=1.1 ..........................
[CV] ........... alpha=1e-05, max_iter=100, epsilon=1.1, total=   0.4s
[CV] alpha=1e-05, max_iter=100, epsilon=1.1 ..........................
[CV] ........... alpha=1e-05, max_iter=100, epsilon=1.1, total=   0.7s
[CV] alpha=1e-05, max_iter=100, epsilon=1.1 ..........................
[CV] ........... alpha=1e-05, max_iter=100, epsilon=1.1, total=   1.3s
[CV] alpha=1e-05, max_iter=1000, epsilon=1.1 .........................
[CV] .......... alpha=1e-05, max_iter=1000, epsilon=1.1, total=   0.4s
[CV] alpha=1e-05, max_iter=1000, epsilon=1.1 .........................
[CV] .......... alpha=1e-05, max_iter=1000, epsilon=1.1, total=   0.7s
[CV] alpha=1e-05, max_iter=1000, epsilon=1.1 .........................
[CV] .

[CV] ........ alpha=1e-05, max_iter=10000, epsilon=1000, total=   1.8s
[CV] alpha=1e-05, max_iter=10, epsilon=10000 .........................
[CV] .......... alpha=1e-05, max_iter=10, epsilon=10000, total=   0.6s
[CV] alpha=1e-05, max_iter=10, epsilon=10000 .........................
[CV] .......... alpha=1e-05, max_iter=10, epsilon=10000, total=   0.6s
[CV] alpha=1e-05, max_iter=10, epsilon=10000 .........................
[CV] .......... alpha=1e-05, max_iter=10, epsilon=10000, total=   0.6s
[CV] alpha=1e-05, max_iter=100, epsilon=10000 ........................
[CV] ......... alpha=1e-05, max_iter=100, epsilon=10000, total=   1.7s
[CV] alpha=1e-05, max_iter=100, epsilon=10000 ........................
[CV] ......... alpha=1e-05, max_iter=100, epsilon=10000, total=   2.3s
[CV] alpha=1e-05, max_iter=100, epsilon=10000 ........................
[CV] ......... alpha=1e-05, max_iter=100, epsilon=10000, total=   1.0s
[CV] alpha=1e-05, max_iter=1000, epsilon=10000 .......................
[CV] .

[CV] ........ alpha=0.0001, max_iter=10000, epsilon=100, total=   1.7s
[CV] alpha=0.0001, max_iter=10000, epsilon=100 .......................
[CV] ........ alpha=0.0001, max_iter=10000, epsilon=100, total=   1.1s
[CV] alpha=0.0001, max_iter=10000, epsilon=100 .......................
[CV] ........ alpha=0.0001, max_iter=10000, epsilon=100, total=   2.8s
[CV] alpha=0.0001, max_iter=10, epsilon=1000 .........................
[CV] .......... alpha=0.0001, max_iter=10, epsilon=1000, total=   0.6s
[CV] alpha=0.0001, max_iter=10, epsilon=1000 .........................
[CV] .......... alpha=0.0001, max_iter=10, epsilon=1000, total=   0.7s
[CV] alpha=0.0001, max_iter=10, epsilon=1000 .........................
[CV] .......... alpha=0.0001, max_iter=10, epsilon=1000, total=   0.4s
[CV] alpha=0.0001, max_iter=100, epsilon=1000 ........................
[CV] ......... alpha=0.0001, max_iter=100, epsilon=1000, total=   2.8s
[CV] alpha=0.0001, max_iter=100, epsilon=1000 ........................
[CV] .

...

[CV] .............. alpha=100, max_iter=1000, epsilon=2, total=   1.5s
[CV] alpha=100, max_iter=1000, epsilon=2 .............................
[CV] .............. alpha=100, max_iter=1000, epsilon=2, total=   0.9s
[CV] alpha=100, max_iter=10000, epsilon=2 ............................
[CV] ............. alpha=100, max_iter=10000, epsilon=2, total=   0.5s
[CV] alpha=100, max_iter=10000, epsilon=2 ............................
[CV] ............. alpha=100, max_iter=10000, epsilon=2, total=   1.5s
[CV] alpha=100, max_iter=10000, epsilon=2 ............................
[CV] ............. alpha=100, max_iter=10000, epsilon=2, total=   0.9s
[CV] alpha=100, max_iter=10, epsilon=10 ..............................
[CV] ............... alpha=100, max_iter=10, epsilon=10, total=   0.3s
[CV] alpha=100, max_iter=10, epsilon=10 ..............................
[CV] ............... alpha=100, max_iter=10, epsilon=10, total=   0.3s
[CV] alpha=100, max_iter=10, epsilon=10 ..............................
[CV] .

[CV] ............ alpha=1000, max_iter=100, epsilon=1.1, total=   0.7s
[CV] alpha=1000, max_iter=1000, epsilon=1.1 ..........................
[CV] ........... alpha=1000, max_iter=1000, epsilon=1.1, total=   0.4s
[CV] alpha=1000, max_iter=1000, epsilon=1.1 ..........................
[CV] ........... alpha=1000, max_iter=1000, epsilon=1.1, total=   1.2s
[CV] alpha=1000, max_iter=1000, epsilon=1.1 ..........................
[CV] ........... alpha=1000, max_iter=1000, epsilon=1.1, total=   0.7s
[CV] alpha=1000, max_iter=10000, epsilon=1.1 .........................
[CV] .......... alpha=1000, max_iter=10000, epsilon=1.1, total=   0.4s
[CV] alpha=1000, max_iter=10000, epsilon=1.1 .........................
[CV] .......... alpha=1000, max_iter=10000, epsilon=1.1, total=   1.4s
[CV] alpha=1000, max_iter=10000, epsilon=1.1 .........................
[CV] .......... alpha=1000, max_iter=10000, epsilon=1.1, total=   0.7s
[CV] alpha=1000, max_iter=10, epsilon=2 ..............................
[CV] .

[CV] .......... alpha=1000, max_iter=100, epsilon=10000, total=   0.6s
[CV] alpha=1000, max_iter=100, epsilon=10000 .........................
[CV] .......... alpha=1000, max_iter=100, epsilon=10000, total=   1.1s
[CV] alpha=1000, max_iter=100, epsilon=10000 .........................
[CV] .......... alpha=1000, max_iter=100, epsilon=10000, total=   1.4s
[CV] alpha=1000, max_iter=1000, epsilon=10000 ........................
[CV] ......... alpha=1000, max_iter=1000, epsilon=10000, total=   0.6s
[CV] alpha=1000, max_iter=1000, epsilon=10000 ........................
[CV] ......... alpha=1000, max_iter=1000, epsilon=10000, total=   1.1s
[CV] alpha=1000, max_iter=1000, epsilon=10000 ........................
[CV] ......... alpha=1000, max_iter=1000, epsilon=10000, total=   1.5s
[CV] alpha=1000, max_iter=10000, epsilon=10000 .......................
[CV] ........ alpha=1000, max_iter=10000, epsilon=10000, total=   0.6s
[CV] alpha=1000, max_iter=10000, epsilon=10000 .......................
[CV] .

[Parallel(n_jobs=1)]: Done 648 out of 648 | elapsed: 12.1min finished


4.710401073583229
